In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

In [2]:
with open('/Users/Alex/.secret/yelp_api.json') as f:   #use your path here!
    login = json.load(f)
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s=5.0)

In [3]:
# set API call parameters 
LOCATION = 'Stamford,CT'
TERM = 'Sushi'

In [4]:
# Specifying JSON_FILE filename (can include a folder)
# include the search terms in the filename
JSON_FILE = "Data/results_in_progress_Stamford_sushi.json"
JSON_FILE

'Data/results_in_progress_Stamford_sushi.json'

In [23]:
def create_json_file(JSON_FILE,  delete_if_exists=False):
    ## Check if JSON_FILE exists
    file_exists = os.path.isfile(JSON_FILE)
    ## If it DOES exist:
    if file_exists == True:
        ## Check if user wants to delete if exists
        if delete_if_exists==True:
            print(f"[!] {JSON_FILE} already exists. Deleting previous file...")
            ## delete file and confirm it no longer exits.
            os.remove(JSON_FILE)
            ## Recursive call to function after old file deleted
            create_json_file(JSON_FILE,delete_if_exists=False)
        else:
            print(f"[i] {JSON_FILE} already exists.")            
    ## If it does NOT exist:
    else:
        ## INFORM USER AND SAVE EMPTY LIST
        print(f"[i] {JSON_FILE} not found. Saving empty list to new file.")
        ## CREATE ANY NEEDED FOLDERS
        # Get the Folder Name only
        folder = os.path.dirname(JSON_FILE)
        ## If JSON_FILE included a folder:
        if len(folder)>0:
            # create the folder
            os.makedirs(folder,exist_ok=True)
        ## Save empty list to start the json file
        with open(JSON_FILE,'w') as f:
            json.dump([],f)  

In [24]:
## Create a new empty json file (exist the previous if it exists)
create_json_file(JSON_FILE, delete_if_exists=True)
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
## How many results total?
total_results = results['total']
## How many did we get the details for?
results_per_page = len(results['businesses'])
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

[!] Data/results_in_progress_Stamford_sushi.json already exists. Deleting previous file...
[i] Data/results_in_progress_Stamford_sushi.json not found. Saving empty list to new file.
- 0 previous results found.


43

In [25]:
for i in tqdm_notebook( range(1,n_pages+1)):
    
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    
    if (n_results + results_per_page) > 1000:
        print('Exceeded 1000 api calls. Stopping loop.')
        break
    
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
    # display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)
    
    time.sleep(.2)

  0%|          | 0/43 [00:00<?, ?it/s]

In [26]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,uEd4N5ZCUMwqjqE1NiSv2g,kiku-sushi-japanese-cuisine-stamford,Kiku Sushi Japanese Cuisine,https://s3-media3.fl.yelpcdn.com/bphoto/Fo6o-h...,False,https://www.yelp.com/biz/kiku-sushi-japanese-c...,97,"[{'alias': 'japanese', 'title': 'Japanese'}]",4.0,"{'latitude': 41.09526003776654, 'longitude': -...","[delivery, pickup]",$$,"{'address1': '1074 Hope St', 'address2': '', '...",+12036090388,(203) 609-0388,3744.619539
1,ztgVu6dExhiHpNqNkEfspg,kashi-stamford-stamford,Kashi - Stamford,https://s3-media3.fl.yelpcdn.com/bphoto/9x4wO7...,False,https://www.yelp.com/biz/kashi-stamford-stamfo...,317,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.0,"{'latitude': 41.0542365163565, 'longitude': -7...","[delivery, pickup]",$$,"{'address1': '131 Summer St', 'address2': None...",+12038909995,(203) 890-9995,5223.544659
2,D2zKZ6PnmKnFDiz03LuOsQ,sushi-soba-old-greenwich,Sushi Soba,https://s3-media3.fl.yelpcdn.com/bphoto/oVTxp_...,False,https://www.yelp.com/biz/sushi-soba-old-greenw...,165,"[{'alias': 'sushi', 'title': 'Sushi Bars'}]",4.5,"{'latitude': 41.044211337885, 'longitude': -73...","[delivery, pickup]",$$,"{'address1': '1345 E Putnam Ave', 'address2': ...",+12039900888,(203) 990-0888,6008.899556
3,18takKdKbeo35NZl3tdW7g,tsuki-sushi-greenwich,Tsuki Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/vtWKaL...,False,https://www.yelp.com/biz/tsuki-sushi-greenwich...,36,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 41.0305, 'longitude': -73.627371}","[delivery, pickup, restaurant_reservation]",NaN,"{'address1': '18 West Putnum Ave', 'address2':...",+12033402588,(203) 340-2588,9155.705171
4,ITgijxOfSftBwhZ0KWAOmg,bentto-sushi-stamford,Bentto Sushi,https://s3-media3.fl.yelpcdn.com/bphoto/Su3az1...,False,https://www.yelp.com/biz/bentto-sushi-stamford...,6,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 41.05354, 'longitude': -73.54036}","[delivery, pickup]",NaN,"{'address1': '300 Main St', 'address2': None, ...",+12035941990,(203) 594-1990,5332.080482


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
837,RPmhoVifrJt4gA3fY-_qqw,the-milleridge-inn-jericho,The Milleridge Inn,https://s3-media2.fl.yelpcdn.com/bphoto/5Hjels...,False,https://www.yelp.com/biz/the-milleridge-inn-je...,729,"[{'alias': 'tradamerican', 'title': 'American ...",2.5,"{'latitude': 40.791561, 'longitude': -73.53775...",[delivery],$$$,"{'address1': '585 N Broadway', 'address2': '',...",+15169312201,(516) 931-2201,34125.746718
838,41l-WTeRkc6dy6Zss9MG4w,six-corners-marketplace-new-york,Six Corners Marketplace,https://s3-media1.fl.yelpcdn.com/bphoto/icaMDv...,False,https://www.yelp.com/biz/six-corners-marketpla...,30,"[{'alias': 'delis', 'title': 'Delis'}, {'alias...",3.0,"{'latitude': 40.80125, 'longitude': -73.9681801}","[delivery, pickup]",NaN,"{'address1': '2755 Broadway', 'address2': '', ...",,,47291.975370
839,DeAR2mVVVFAPV3luScb_YQ,king-kullen-manhasset,King Kullen,https://s3-media3.fl.yelpcdn.com/bphoto/gBRyAY...,False,https://www.yelp.com/biz/king-kullen-manhasset...,37,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",2.5,"{'latitude': 40.7932534, 'longitude': -73.6932...",[],$$,"{'address1': '1430 Northern Blvd', 'address2':...",+15163652473,(516) 365-2473,35782.557046
840,vdmLsd-_-EQ7W6TnvfYdAQ,talias-steakhouse-and-bar-new-york-4,Talia's Steakhouse & Bar,https://s3-media1.fl.yelpcdn.com/bphoto/9yYuK7...,False,https://www.yelp.com/biz/talias-steakhouse-and...,393,"[{'alias': 'steak', 'title': 'Steakhouses'}, {...",3.0,"{'latitude': 40.79201414780005, 'longitude': -...","[delivery, pickup]",$$$,"{'address1': '668 Amsterdam Ave', 'address2': ...",+12125803770,(212) 580-3770,48255.070573
841,SL1bTm69iMihPLkEQD7CzQ,jersey-mikes-subs-fort-lee,Jersey Mike's Subs,https://s3-media1.fl.yelpcdn.com/bphoto/LwTGIq...,False,https://www.yelp.com/biz/jersey-mikes-subs-for...,85,"[{'alias': 'hotdogs', 'title': 'Fast Food'}, {...",3.5,"{'latitude': 40.85612, 'longitude': -73.96776}","[delivery, pickup]",NaN,"{'address1': '2151 Lemoine Ave', 'address2': N...",+12012921987,(201) 292-1987,43225.975968


In [28]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [29]:
# save the final results to a compressed csv
final_df.to_csv('Data/final_results_Stamford_sushi.csv.gz', compression='gzip',index=False)